In [ ]:
import os

In [22]:
def set_api_key(filepath: str = '.env'):
    """
    Set the OpenAI API key
    """
    with open(filepath, 'r') as f:
        for line in f:
            if line.startswith("OPENAI_API_KEY"):
                os.environ['OPENAI_API_KEY'] = line.strip().split("=")[-1].strip("\"")

    assert os.environ['OPENAI_API_KEY'] is not None or os.environ['OPENAI_API_KEY'] != ""

def read_system_prompt(filepath: str = "system_promt.txt"):
    pass

set_api_key()

In [12]:
from openai import OpenAI

client = OpenAI()

tools = [
  {
      "type": "function",
      "function": {
          "name": "get_weather",
          "parameters": {
              "type": "object",
              "properties": {
                  "location": {"type": "string"}
              },
          },
      },
  }
]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "What is the weather like in Austin, TX today"
        }
    ],
    tools=tools
)

print(completion.choices[0].message.tool_calls)

[ChatCompletionMessageToolCall(id='call_nJ2xwYVLZyTRujFSZZx5S1fS', function=Function(arguments='{"location":"Austin, TX"}', name='get_weather'), type='function')]
